In [1]:
import requests
# Import the json library
import json
import pandas as pd
from IPython.core.display import clear_output

from warnings import warn
from time import time
from time import sleep
from random import randint

from bs4 import BeautifulSoup

import re

#https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/

In [2]:
start = "https://games.crossfit.com/scores/leaderboard.php?stage=5&sort=0&division=1&region=0&regional=5&numberperpage=60&userid=0&competition=0&frontpage=0&expanded=1&year=13&full=1&showtoggles=0&hidedropdowns=1&showathleteac=1&athletename=&scaled=0"
#start = base + "open/2016/leaderboards"
parameters = {"page":0,"division":1,"region":2}

In [3]:
#region = 0 is worldwide
divisionList = [1,2] #TODO: get division from table not hardcode

In [4]:
headers = {
    'User-Agent': 'Jessica Rudd, jrudd1@students.kennesaw.edu',
    'From': 'jrudd1@students.kennesaw.edu'
}

In [5]:
df = pd.DataFrame()

yearList = []
competitionTypeList = []

affilliateNameList= []
ageList = []
competitorIdList = []
competitorNameList = []
genderList = []
heightList = []
professionList = []

divisionIdList = []
professionList = []
regionIdList = []
regionNameList = [] ###
teamCaptainList = []
weightList = []

scoreOverallList= []
scoreOverallRank= []

scoreOneList = []
scoreTwoList = []
scoreThreeList = []
scoreFourList = []
scoreFiveList = []
scoreSixList = []

scoreOneDisplayList = []
scoreTwoDisplayList = []
scoreThreeDisplayList = []
scoreFourDisplayList = []
scoreFiveDisplayList = []
scoreSixDisplayList = []

scoreOneRankList = []
scoreTwoRankList = []
scoreThreeRankList = []
scoreFourRankList = []
scoreFiveRankList = []
scoreSixRankList = []

errorList = []

In [6]:
start_time = time()
requestcount = 0
totalcount = 0 

In [7]:
#with help from https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/

class HTMLTableParser:

    def parse_url(self, url, parameters):
        response = requests.get(url,headers=headers,params=parameters)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            column_names.append("") #TODO: this temp fix
            #raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        #print(df.head())
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
        #print(df.head())
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        # this added to clean
        #TODO: fix this correctly
       # df.drop(df.columns[0:7],axis=1,inplace=True)
        df.columns = column_names
        return df

In [8]:
#need how to parse \n1 (143)\n Validated by: V... into rank 1 score 143
#need to iterate through list using values
#create function returning two lists and assign as tuples
def parseCrossString(myString):
    return myString.split("\n",1)[1].split(" ",1)[0],re.sub('[()]', '', myString.split("\n",1)[1].split(" ",1)[1].split("\n")[0])

In [ ]:
#for each region need to iterate through and get page count
for division in divisionList:
    
    # need to get regions to iterate through
    region = {}
    parameters = {"page":0,"division":division}
    response = requests.get(start,headers = headers)
    soup = BeautifulSoup(response.text, 'lxml')
    mydivs = soup.findAll('div', {'class': 'form-item form-type-select form-item-division '})
    for div in mydivs:
        if div.label.text == 'Region ':
            for children in div.findAll('option'):
                if children.text.strip() != 'Worldwide':
                    region[children['value']] =children.text.strip()

    #need number of pages per region
    for regionkey, regionvalue in region.items():
        
        requestcount =0
        
        #need way to get max page number eg 237 will be by region
        parameters = {"page":0,"division":division,"region":regionkey}
        response = requests.get(start, params=parameters, headers = headers)
        soup = BeautifulSoup(response.text, 'lxml')
    
        mybuttons = soup.findAll('a', {'class': 'button'})
        if len(mybuttons) > 0:
            numPages = int(mybuttons[0].text)
        else:
            numPages = 1
    
        #iterate through each page passing in params
        for page in range(1,2):
            
            try:
                parameters = {"page":page,"division":division,"region":regionkey}
                
                            # Monitor the requests
                requestcount += 1
                totalcount += 1
                elapsed_time = time() - start_time
                print('Request:{} of {} for Division {} and Region {}; Errors: {} Frequency: {} requests/s for {} total requests'.format(requestcount, numPages, division, regionvalue, len(errorList),requestcount/elapsed_time, totalcount))
                clear_output(wait = True)
            
                hp = HTMLTableParser()
                table = hp.parse_url(start,parameters)[0][1]

                #append to competitor list
                competitorNameList.append(table['Competitor'].values)

                #need to iterate through workout columns and add to lists
                for counter,i in enumerate(table[[col for col in table.columns if 'Workout' in col]].columns):
                    ranklist,scorelist = map(list,zip(*[parseCrossString(value) for value in table[i]]))
                    print(counter)
                    if counter==0:
                        scoreOneList.append(scorelist)
                        scoreOneRankList.append(ranklist)
                    if counter==1:
                        scoreTwoList.append(scorelist)
                        scoreTwoRankList.append(ranklist)
                    if counter==2:
                        scoreThreeList.append(scorelist)
                        scoreThreeRankList.append(ranklist)
                    if counter==3:
                        scoreFourList.append(scorelist)
                        scoreFourRankList.append(ranklist)
                    if counter==4:
                        scoreFiveList.append(scorelist)
                        scoreFiveRankList.append(ranklist)
                    if counter==5:
                        scoreSixList.append(scorelist)
                        scoreSixRankList.append(ranklist)

                records = len(table['Competitor'])
                yearList.append(['2013'] * records) #TODO: hardcoded because not in this version of API
                competitionTypeList.append(['open'] * records) #TODO: hardcoded because not in this version of API

                divisionIdList.append([division] * records)
                regionIdList.append([regionkey] * records)
                regionNameList.append([regionvalue] * records)

                # Pause the loop
                sleep(randint(8,15)/100)
                
            except:
                errorList.append([parameters]+[response.content])

Request:1 of 1 for Division 11 and Region Africa; Errors: 0 Frequency: 0.9255798578055769 requests/s for 1 total requests
0
1
2
3
4
Request:1 of 1 for Division 11 and Region Asia; Errors: 0 Frequency: 0.5289725811789205 requests/s for 2 total requests
0
1
2
3
4
Request:1 of 3 for Division 11 and Region Australia; Errors: 0 Frequency: 0.3246253515262052 requests/s for 3 total requests
0
1
2
3
4
Request:1 of 2 for Division 11 and Region Canada East; Errors: 0 Frequency: 0.2336368832982012 requests/s for 4 total requests
0
1
2
3
4
Request:1 of 1 for Division 11 and Region Canada West; Errors: 0 Frequency: 0.1956835176811551 requests/s for 5 total requests
0
1
2
3
4


In [ ]:
allData = pd.DataFrame({'yearList':[y for x in yearList for y in x],
                'competitionType':[y for x in competitionTypeList for y in x],
                'regionId':[y for x in regionIdList for y in x],
                'regionName':[y for x in regionNameList for y in x],
                'divisionId':[y for x in divisionIdList for y in x],
                'competitorName':[y for x in competitorNameList for y in x],
                
                'scoreOne':[y for x in scoreOneList for y in x],
                'scoreTwo':[y for x in scoreTwoList for y in x],
                'scoreThree':[y for x in scoreThreeList for y in x],
                'scoreFour':[y for x in scoreFourList for y in x],
                'scoreFive':[y for x in scoreFiveList for y in x],

                'scoreOneRank':[y for x in scoreOneRankList for y in x],
                'scoreTwoRank':[y for x in scoreTwoRankList for y in x],
                'scoreThreeRank':[y for x in scoreThreeRankList for y in x],
                'scoreFourRank':[y for x in scoreFourRankList for y in x],
                'scoreFiveRank':[y for x in scoreFiveRankList for y in x]
             }
)

In [1]:
allData

NameError: name 'scoreThreeList' is not defined

In [15]:
allData.to_csv('data2013.csv')